<a href="https://colab.research.google.com/github/zahr-eddine-elbouzidi/tasks_nlp_transformers/blob/main/tasks_nlp_using_huggingface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**NLP Tasks with Transformers using HuggingFace**

*   Sentiment analysis
*   Text generation
*   Name entity recognition (NER)
*   Question answering
*   Filling masked text
*   Summarization
*   Translation
*   Feature extraction




###**Import packages**###

In [20]:
#!pip install transformers sentencepiece

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


###**DataSet**##

In [26]:
array_of_texts = [      
            ["I love my job in Morocco", "English"],
            ["my name is Ahmed I hate my job","English"],
            ["في المغرب أحب عملي جدا" , "Arabic"],
            ["لا أحب عملي بالمرة", "Arabic"],
            ["J'aime mon travail au Maroc", "French"],
            ["Je n'aime pas mon travail", "French"]
]

print(array_of_texts)

[['I love my job in Morocco', 'English'], ['my name is Ahmed I hate my job', 'English'], ['في المغرب أحب عملي جدا', 'Arabic'], ['لا أحب عملي بالمرة', 'Arabic'], ["J'aime mon travail au Maroc", 'French'], ["Je n'aime pas mon travail", 'French']]


In [23]:
from transformers import pipeline,AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("akhooli/gpt2-small-arabic")
model = AutoModel.from_pretrained("akhooli/gpt2-small-arabic")

Some weights of the model checkpoint at akhooli/gpt2-small-arabic were not used when initializing GPT2Model: ['lm_head.weight']
- This IS expected if you are initializing GPT2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [25]:
my_classifier_using_pipeline = pipeline('sentiment-analysis' ,  model='akhooli/xlm-r-large-arabic-sent')

Some weights of the model checkpoint at akhooli/xlm-r-large-arabic-sent were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


###**Preprocessing data**

In [26]:
import string
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')
punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ''' + string.punctuation  #all punctuations 

def preprocessing_data(text , type):
    #remove punctuations
 
  
    if type == "Arabic":

   
      #remove longation and stopwords
      text = re.sub("[إأآا]", "ا", text)
      text = re.sub("ى", "ي", text)
      text = re.sub("ؤ", "ء", text)
      text = re.sub("ئ", "ء", text)
      text = re.sub("ة", "ه", text)
      text = re.sub("گ", "ك", text)

    
    elif type == "English":

      text = text.lower()
      text = ''.join([item for item in text if item not in punctuations ])
      
 
    return text
  
 
 

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


####**Text classification (Sentiments Analysis) - English, French and Arabic languages**

In [27]:
for text,language in array_of_texts:
  res = my_classifier_using_pipeline(preprocessing_data(text , language))[0]
  res['label'] = "Positive" if res['label'] == 'LABEL_2' else ("Negative" if res['label'] == 'LABEL_1'  else "Mixed")
  print("'{}' is {} {} text with occuracy = {}%".format(text,res['label'],language,round(res['score']*100, 2)))


'I love my job' is Positive English text with occuracy = 64.56%
'I hate my job' is Negative English text with occuracy = 78.19%
'في المغرب أحب عملي جدا' is Positive Arabic text with occuracy = 55.78%
'لا أحب عملي بالمرة' is Negative Arabic text with occuracy = 81.6%
'J'aime mon travail' is Positive French text with occuracy = 47.54%
'Je n'aime pas mon travail' is Negative French text with occuracy = 95.15%


###**Text generation (English, French and Arabic languages)**

In [1]:
from transformers import pipeline, set_seed
generator_english_text = pipeline('text-generation', model='gpt2')
generator_arabic_text = pipeline('text-generation', model='akhooli/gpt2-small-arabic')
generator_french_text = pipeline('text-generation', model='dbddv01/gpt2-french-small')
set_seed(42)

2021-07-23 14:15:23.446378: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [3]:
for text , lang in array_of_texts:
  if lang == "Arabic":
    print(generator_arabic_text(text , max_length=30, num_return_sequences=5))
  elif lang == "English":
    print(generator_english_text(text , max_length=30, num_return_sequences=5))
  else:
    print(generator_french_text(text , max_length=30, num_return_sequences=5))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'I love my job! I hope my family gets an award at the end of this thing." He paused for a moment before speaking again before closing the'}, {'generated_text': "I love my job and I'm enjoying my job and I'm glad it was done by a guy like this who gives his best in this area of"}, {'generated_text': 'I love my job," said the manager, who goes by her first name.\n\nThat is until this week when Lourdes is due to'}, {'generated_text': 'I love my job! I am really happy.\n\nThis article originally appeared in the November 2016 issue of Fortune.'}, {'generated_text': 'I love my job, so why would I want to join a multinational company when I could give my kids a better job?" Her answer was a res'}]


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'I hate my job and it\'s too much." — Soren Kierkegaard, in "The Desperately Seeking Freedom," 2004\n\nBut'}, {'generated_text': 'I hate my job," he said. "We\'ve lost so many good workers. It\'s time for us to move on."\n\nIt\'s'}, {'generated_text': "I hate my job because of what I'm doing and I miss my parents and my grandmother. That didn't happen to me so I'll take what"}, {'generated_text': "I hate my job. You should work hard and your wife will always be there for you. The biggest hurdle you'll face is how much you're"}, {'generated_text': "I hate my job. I'm not interested in the fact of getting the job done. I'm just interested in working. I just don't come"}]


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'في المغرب أحب عملي جدا، فحرص على عدم خوض غمار الحرب، حتى إنه عندما وصل إلى الجزائر كتب إلى الشيخ أحمد بن صالح الجزائري في كتابه'}, {'generated_text': 'في المغرب أحب عملي جدا وأتقن بعض اللغات من اللغات الفرنسية التي لم تكن معروفة بعد. بعد عدة أشهر من هذه الأبحاث قرر أن يصبح كاتب مقال كتب'}, {'generated_text': 'في المغرب أحب عملي جدا ثم قدم استقالته سنة 1955 . كان على رأس السلطة في المغرب أن يقرر أن يمنح الجنسية المغربية. فأقر المغرب عند عودته إلى'}, {'generated_text': 'في المغرب أحب عملي جدا وهو فن الخط، وكان يقول له: كنت أحتاج إلى أن أحتاج إلى أن أشهد من هذا الفن. وكان يقول'}, {'generated_text': 'في المغرب أحب عملي جدا، فكان له ابن وحيد اسمه علي بن أبي طالب، وهو علي بن أبي طالب، وعائشة. قال ابن رجب: وحدث'}]


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'لا أحب عملي بالمرة في كل صباح، إذ لم يكن قادرا على أن يرسم المشاهد إلا بتصرفات دقيقة، حتى في حالة من اليأس أو الشر.'}, {'generated_text': 'لا أحب عملي بالمرة من حيث الشكل. عندما بلغ سن الرشد بقليل، حاول أن يجد نفسه في أحد الأيام، فقد كانت لديه رؤية واضحة، لذا'}, {'generated_text': 'لا أحب عملي بالمرة، وهذا أمر بالغ الأهمية بالنسبة لنا عندما يقول: إنما كنت أسمع كل كلمة فقد أردت أن تقوم بكل هذه الكلمة،'}, {'generated_text': 'لا أحب عملي بالمرة. لا يوجد شئ سوى الإشارة إلى أنه بعد وفاة الشيخ، لم يترك في المكان أي طعام رسمي. وفي هذه الأثناء، أمر'}, {'generated_text': 'لا أحب عملي بالمرة رغم أنها كانت بعيدة عن نفسها ولم تعد تعمل من أجل هذه الحياة فقد كانت تعاني من آلام وتورم وسؤر من مرض'}]


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "J'aime mon travail. Si on ne me touche pas de la peine, c'est une bonne chose. » (« I don't to be"}, {'generated_text': "J'aime mon travail. Je me suis tourné vers la ville de Toulouse, capitale de l'Union européenne, et je me suis tourné à Toulouse"}, {'generated_text': "J'aime mon travail et m'ennuient les critiques et les commentaires de la presse spécialisée,,,,. Après l'avoir demandé, je lui ai répondu"}, {'generated_text': "J'aime mon travail ; j'aime les films et la peinture ». L'une des raisons de travailler sur l'écliptique est qu"}, {'generated_text': "J'aime mon travail et je m'en sens comme une femme et c'est de la maison. »Le 8 janvier 1934, la version"}]
[{'generated_text': "Je n'aime pas mon travail. »À la suite de ce nouvel échec commercial, Walt Disney est obligé de lancer la production du film La Belle"}, {'generated_text': "Je n'aime pas mon travail, j'ai trouvé le plaisir de vivre, ça meesprit. »Le club s'impose de nouveau contre"}, {'generated_

###**Name entity recognition (NER) English, French and Arabic languages**

In [24]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

ner_english_recognition = pipeline("ner", model="dslim/bert-base-NER", tokenizer="dslim/bert-base-NER")
ner_arabic_recognition = pipeline("ner", model="hatmimoha/arabic-ner", tokenizer="hatmimoha/arabic-ner")
ner_french_recognition = pipeline("ner", model="gilf/french-postag-model", tokenizer="gilf/french-postag-model")

Downloading:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/712M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/996k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [27]:
for text , lang in array_of_texts:
  if lang == "Arabic":
    print(ner_arabic_recognition(text))
  elif lang == "English":
    print(ner_english_recognition(text))
  else:
    print(ner_french_recognition(text))

[{'entity': 'B-LOC', 'score': 0.99983495, 'index': 6, 'word': 'Morocco', 'start': 17, 'end': 24}]
[{'entity': 'B-PER', 'score': 0.9895351, 'index': 4, 'word': 'Ahmed', 'start': 11, 'end': 16}]
[{'entity': 'LABEL_12', 'score': 0.99964076, 'index': 1, 'word': 'في', 'start': 0, 'end': 2}, {'entity': 'LABEL_4', 'score': 0.99920195, 'index': 2, 'word': 'المغرب', 'start': 3, 'end': 9}, {'entity': 'LABEL_12', 'score': 0.99957824, 'index': 3, 'word': 'أحب', 'start': 10, 'end': 13}, {'entity': 'LABEL_12', 'score': 0.99991935, 'index': 4, 'word': 'عملي', 'start': 14, 'end': 18}, {'entity': 'LABEL_12', 'score': 0.99992007, 'index': 5, 'word': 'جدا', 'start': 19, 'end': 22}]
[{'entity': 'LABEL_12', 'score': 0.9999264, 'index': 1, 'word': 'لا', 'start': 0, 'end': 2}, {'entity': 'LABEL_12', 'score': 0.99991745, 'index': 2, 'word': 'أحب', 'start': 3, 'end': 6}, {'entity': 'LABEL_12', 'score': 0.99994904, 'index': 3, 'word': 'عملي', 'start': 7, 'end': 11}, {'entity': 'LABEL_12', 'score': 0.99994946, '

###**Question answering**

In [33]:
from transformers import pipeline

##### English context
question = "What is the capital of the Morocco?"
context = r"Morocco, a North African country bordering the Atlantic Ocean and Mediterranean Sea, is distinguished by its Berber, Arabian and European cultural influences. Marrakesh’s medina, a mazelike medieval quarter, offers entertainment in its Djemaa el-Fna square and souks (marketplaces) selling ceramics, jewelry and metal lanterns. The capital Rabat’s Kasbah of the Udayas is a 12th-century royal fort overlooking the water."
# Generating an answer to the question in context
qa_english_sentence = pipeline("question-answering")
answer = qa_english_sentence(question=question, context=context)
# Print the answer
print(f"English Question: {question}")
print(f"English Answer: '{answer['answer']}' with accuracy {answer['score']}")

##### French context
qa_french_sentence = pipeline('question-answering', model='fmikaelian/camembert-base-squad', tokenizer='fmikaelian/camembert-base-squad')
question_fr = "Quelle est la capitale du Maroc ?"
context_fr = "Le Maroc est un pays du nord-ouest de l'Afrique. Sa longue côte donnant sur l'océan Atlantique se termine au cap Spartel, limite occidentale du détroit de Gibraltar et où débute le littoral méditerranéen. Au sud du Maroc se trouve le territoire contesté Sahara occidental, revendiqué et contrôlé en grande partie par le Maroc. À l'est et au sud-est, le Maroc est limitrophe de l'Algérie. À l'ouest-sud-ouest et à quelque distance de la côte atlantique se situent les îles Canaries tandis qu'à 672 km à l'ouest-nord-ouest du littoral marocain, on rencontre Madère. Au nord du détroit de Gibraltar se trouve l'Espagne. La capitale administrative est Rabat. Parmi les grandes villes remarquables on trouve Casablanca, Agadir, Fès, Marrakech, Meknès, Tanger, Oujda, Nador."
french_answer = qa_french_sentence(question =  question_fr , context = context_fr)
# Print the answer
print(f"French Question: {question_fr}")
print(f"French Answer: '{french_answer['answer']}' with accuracy {french_answer['score']}")


English Question: What is the capital of the Morocco?
English Answer: 'Rabat’s Kasbah of the Udayas' with accuracy 0.7165274620056152
French Question: Quelle est la capitale du Maroc ?
French Answer: ' Rabat.' with accuracy 0.9795011878013611


###**Filling masked text in English,French and Arabic languages**

In [10]:
from transformers import pipeline
english_fill_mask = pipeline('fill-mask', model='bert-large-uncased-whole-word-masking')
french_fill_mask  = pipeline("fill-mask", model="camembert-base", tokenizer="camembert-base")
arabic_fill_mask = pipeline('fill-mask', model='CAMeL-Lab/bert-base-camelbert-ca')

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/468 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/439M [00:00<?, ?B/s]

Some weights of the model checkpoint at CAMeL-Lab/bert-base-camelbert-ca were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/86.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/305k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [14]:
print(fillmask_english("Hi I'm a [MASK] data scientist."))
print(french_fill_mask("L'examen est <mask>."))
print(arabic_fill_mask("الهدف من الدراسة هو [MASK] ."))

[{'sequence': "hi i'm a computer data scientist.", 'score': 0.48725995421409607, 'token': 3274, 'token_str': 'computer'}, {'sequence': "hi i'm a freelance data scientist.", 'score': 0.044282227754592896, 'token': 15919, 'token_str': 'freelance'}, {'sequence': "hi i'm a research data scientist.", 'score': 0.03349841758608818, 'token': 2470, 'token_str': 'research'}, {'sequence': "hi i'm a data data scientist.", 'score': 0.03199973702430725, 'token': 2951, 'token_str': 'data'}, {'sequence': "hi i'm a math data scientist.", 'score': 0.026594826951622963, 'token': 8785, 'token_str': 'math'}]
[{'sequence': "L'examen est terminé.", 'score': 0.2890775203704834, 'token': 3818, 'token_str': 'terminé'}, {'sequence': "L'examen est ouvert.", 'score': 0.061306796967983246, 'token': 1422, 'token_str': 'ouvert'}, {'sequence': "L'examen est gratuit.", 'score': 0.04366162046790123, 'token': 434, 'token_str': 'gratuit'}, {'sequence': "L'examen est annulé.", 'score': 0.03470311686396599, 'token': 16737, 

###**Summarization**

In [15]:
from transformers import pipeline
summarizer = pipeline("summarization")

Downloading:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

In [19]:
article_summary = """Marrakech, Morocco (CNN)Renewable energy is taking off in Morocco.
In 2014 the country opened the largest wind farm in Africa, valued at $1.4 billion, in the southwest near the city of Tarfaya. Then, in early 2016, it switched on the first facility of the world's largest concentrated solar plant, Noor-1, on the fringe of the Sahara desert. When completed in 2018, it will power one million homes and make Morocco a solar superpower.
And while the country is still heavily reliant on energy imports (90 percent in 2013, according to the World Bank), it plans to generate 40 percent of its energy from renewables by 2020.
Following this road has led to Morocco hosting the UN's 2016 summit on climate change, COP22, in Marrakech. So what lies ahead?
Cutting the energy bill
Morocco ranks seventh in the world in the 2016 Climate Change Performance Index, and is the only non-European country in the top 20.
It's also one of only five countries to have achieved a "sufficient" rating for its efforts to keep warming below 2°C in the Climate Action Tracker (no country has achieved the "role model" rating as of yet, so "sufficient" is currently the best grade)."""

context_fr = "Le Maroc est un pays du nord-ouest de l'Afrique. Sa longue côte donnant sur l'océan Atlantique se termine au cap Spartel, limite occidentale du détroit de Gibraltar et où débute le littoral méditerranéen. Au sud du Maroc se trouve le territoire contesté Sahara occidental, revendiqué et contrôlé en grande partie par le Maroc. À l'est et au sud-est, le Maroc est limitrophe de l'Algérie. À l'ouest-sud-ouest et à quelque distance de la côte atlantique se situent les îles Canaries tandis qu'à 672 km à l'ouest-nord-ouest du littoral marocain, on rencontre Madère. Au nord du détroit de Gibraltar se trouve l'Espagne. La capitale administrative est Rabat. Parmi les grandes villes remarquables on trouve Casablanca, Agadir, Fès, Marrakech, Meknès, Tanger, Oujda, Nador."

print("Summary:",summarizer(article_summary, max_length=130, min_length=30, do_sample=False)[0]['summary_text'])
print("Summary:",summarizer(context_fr, max_length=130, min_length=30, do_sample=False)[0]['summary_text'])

Summary:  Morocco plans to generate 40 percent of its energy from renewables by 2020 . The country is hosting the UN's 2016 summit on climate change, COP22, in Marrakech .
Summary:  Le Maroc est un pays du nord-ouest de l'Afrique . Sa longue côte donnant sur l'océan Atlantique se termine au cap Spartel, limite occidentale du détroit de Gibraltar . Au sud du Maroc se trouve le territorie contesté Sahara occidental .


###**Translation**

In [2]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

article_ar = "السلطات المختصة تعيد فرض قوانين صارمة"

model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")

# translate Arabic to English
tokenizer.src_lang = "ar_AR"
encoded_ar = tokenizer(article_ar, return_tensors="pt")
generated_tokens = model.generate(**encoded_ar, forced_bos_token_id=tokenizer.lang_code_to_id["en_XX"])
tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
# => "The Secretary-General of the United Nations says there is no military solution in Syria."

['The authorities are reintroducing strict rules.']

###**Feature extraction**

In [8]:
from transformers import pipeline, AutoTokenizer

feature_extraction = pipeline('feature-extraction', model="distilroberta-base", tokenizer="distilroberta-base")
features = feature_extraction("i am data scientist")



Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
print(features[0])
print(len(features[0])) #6

6
